Numerical analysis (or data analysis) is what Python is especially good at, and probably why you want to learn Python.  Now we will use a simple project to guide you.  Let's use Python to play music.

In [ ]:
# This cell sets up the notebook.

%matplotlib inline

# Import necessary modules.

import traceback
import os

import numpy as np
import matplotlib.pyplot as plt
from IPython.core.magic import register_cell_magic
from IPython.display import display, Audio

In [ ]:
# This cell sets up helpers for demonstration in the course.
# Before the end of this session, you shouldn't take this as examples.

# Define constants.

sampling_freq = 44100 # 44.1 kHz.

# Define helper functions.

def time_data(duration, rate=sampling_freq):
    return np.linspace(0, duration, num=int(rate*duration))

def sine_data(freq, time, damp=0):
    data = 2**13 * np.sin(2*np.pi * freq * time)
    if damp: # Remove gitters.
        todamp = int(damp * sampling_freq)
        darr = np.arange(todamp)/(todamp-1)
        darr = np.sin(darr * np.pi / 2)
        data[:todamp] *= darr
        data[-todamp:] *= 1-darr
    return data

def note_freq(note):
    """Parse note symbol "a4", "c4", etc. and calculate frequency (in Hz)."""
    if ' ' == note:
        return 0.0
    f0 = 440.0 # A4
    note = note.lower()
    diff = {'a': 0, 'b': 2, 'c': -9, 'd': -7, 'e': -5, 'f': -4, 'g': -2}[note[0]]
    diff += {'#': 1, 'b': -1}.get(note[1], 0)
    diff += 12 * (int(note[-1])-4)
    return f0 * 2 ** (float(diff)/12)

def play_notes(duration, *notes):
    time = time_data(duration, sampling_freq)
    alist = [sine_data(note_freq(n), time) for n in notes]
    data = np.vstack(alist).sum(axis=0)
    display(Audio(data.astype('int16'), rate=sampling_freq))
    return time, data

The above chunk of code allows you to play chords.  For example, the C chord:

In [ ]:
print("Play the C chord for 3 seconds:")
time, data = play_notes(3, 'c4', 'e4', 'g4')

print("Plot the first 1000 time points (around %.0f ms):" % (1.e6/sampling_freq))
plt.plot(time[0:1000], data[0:1000])
plt.xlim(time[0], time[999])
plt.xlabel('time (second)')
plt.ylabel('amplitude')

We will use this as an example to introduce you how to program in Python.  The above code is organized in a compact way to work like a library.  In what follows, to make a clear demonstration, we will expand it to give you a plain sight.

# Variables

Variables are the things that Python uses to keep track of information.  Let's see an example:

In [ ]:
# First, we create some data by calling the helper function "time_data()".
# At the same time, we name it the "time", which is the variable.
time = time_data(10)
# By using the "time" varaible, we can send the data to the tool "plt.plot()",
# and plot the data.  As they are shown, the data are a series of increasing
# values of the same difference.
plt.plot(time)

The example demonstrates a key feature of a variable: It is a handle to access the associated data.  The action (line 3) use the variable "`time`" to name the data returned by `time_data()`, so that we can tell Python to send the data to `plt.plot()`.  Variables give us the ability to name things in Python.  This is a very basic and critical thing in programming.

As an exercise, try to do a similar thing but use another variable.  You should still see a straight line, but the maximum value becomes 12 instead of 10.

**Note**: In Python, everything that follows the "`#`" (pronounced "sharp" or "dial") symbol is taken as comments (code remarks).  They are considered not part of the code, and allow programmers to use natural language to explain the code.  You will expect a lot of them in this notebook.

In [ ]:
other_time = time_data(12)
plt.plot(other_time)

In Python, a variable can be any type of data.  You don't need to specify or know the kind of data before using the variable.  But you do need to have the variable set before use it.

In [ ]:
a_number = 10
print(a_number)
a_word = "advancement"
print(a_word)

print(a_condition) # Won't work since a_condition isn't yet set.

# Numbers

Let's talk about numbers by a simple formula that calculates saving interests.

Say the annual interest rate is 2%.  Although Python cannot allow a percentage number, we know it is simply the real number `0.02`, and write it as

```python
interest_rate = 0.02 # 2% compound interest rate (annualized).
```

In [ ]:
interest_rate = 0.02 # 2% compound interest rate (annualized).
print(interest_rate)

For numbers, modern Python has a neat feature, is to allow you to insert eye candy for better grouping.  The underscore (`_`) can be used to separate the number 1 million by thousand:

```python
principal = 1_000_000
```

Compare to the representation without the underscores:

```python
principal = 1000000
```

The former helps you avoid error.

In [ ]:
principal = 1_000_000 # The money you save monthly.
print(principal)

In [ ]:
print(type(1)) # This is an integer

In [ ]:
print(type(1.0)) # This is a real number (floating-point)

## True Division

Python division (`/`) operator is kind of special and the convenience may surprise programmers from other languages.  The division operator is called the "true division", because the result is always a real number (floating-point).

In [ ]:
print(1/1)
print(type(1/1)) # An integer divided by an integer gives a float!

The good old integer division uses the "floor division" operator, `//`.  As the name suggests, it always returns the floor:

In [ ]:
# No tie breaking for integer.
print(2//3)

In [ ]:
# No tie breaking for real number, either.
print(2.0//3.0)

## Arithmatic

The named variables help us to express calculation in a clear way:

```python
interest = principal*(1+interest_rate/12)**12 - principal # The interest rolls in monthly.
```

The bank uses a compound interest rate, and we want to know how much money we eventually get in the whole year.  The interest is evaluated every month, so the rate needs to be divided by 12.  It's the compound rate, so we multiply the principal by `(1+interest_rate/12)` 12 times.

The formula uses the most common opeators in Python: `+` (addition), `-` (subtraction), `*` (multiplication), `/` (division).  It also uses the power operator `**`.  The subtraction is worth a note: it always returns a real number and doesn't automatically round to integer:

In [ ]:
interest = principal*(1+interest_rate/12)**12 - principal # The interest rolls in monthly.
print(interest)

# Format Numbers in Strings

Numbers and strings are the most straight-forward variables.  By using the high-school algebra we immedinately make sense of the following examples.

In [ ]:
formatted = "Interest is {:,.0f} with {:,.0f} at rate {:.1%}.".format(interest, principal, interest_rate)
print(formatted)

Let's see what happens in the above statement.  The first part is the template string (also called formatting string):

```python
"Interest is {:,.0f} with {:,.0f} at rate {:.1%}."
```

The second part uses a method attached to the template string to fill in the data:

```python
.format(interest, principal, interest_rate))
```

In the end, Jupyter shows the formatted string:

```
Interest is 20,184 with 1,000,000 at rate 2.0%.
```

The substring enclosed by the pair of curled braces (`{}`) in the template string specifies how the associated variable is formatted.  We use two of them in the above example:

* `{:,.0f}` says to format the real number using fixed-point notation, use `,` to group by thousand, and show no (0) digit after decimal points.
* `{:.1%}` says to format the real number as percentage, and show 1 digit after decimal point.

But if you don't bother the detailed formatting, it's fine to simply write `{}` as placeholders.  The result is only slightly less clear but you still get it.

In [ ]:
print("Interest is {} with {} at rate {}.".format(interest, principal, interest_rate))

## Project Step 1: Music Notes

The C chord consists of three notes: C4, E4, and G4.  To synthesis the sound, we first need to know the frequencies of the notes.  The formula is:

\begin{equation*}
f = 2 ^ {\frac{d}{12}} \times f_0
\end{equation*}

where $f_0$ is the frequency of the reference note and $d$ is the difference to the reference note.  If we choose A4 as the reference note, and use its frequency 440 Hz, it's straight-forward to calculate the frequencies for the notes we want.

In [ ]:
# Define the reference frequency, and calculate the other 
# frequencies for C4, E4, and G4.
ref_freq = 440.0 # The reference frequency uses A4.

c4_freq = ref_freq * 2 ** (-9.0/12) # A4 - C4 = 9
print("Frequency for C4: {:.2f} Hz".format(c4_freq))
e4_freq = ref_freq * 2 ** (-5.0/12) # A4 - E4 = 5
print("Frequency for E4: {:.2f} Hz".format(e4_freq))
g4_freq = ref_freq * 2 ** (-2.0/12) # A4 - G4 = 2
print("Frequency for G4: {:.2f} Hz".format(g4_freq))

# Values and Containers

In Python, variables can be broadly categorized into two kinds: a mere value and a group of values.  The latter is usually much more useful than the former because repetive work is what computers do especially better than humans.

## List

List is an example of containers.  You can put a sequence of numbers in a list:

In [ ]:
W_inch = 44 # width
H_inch = 32 # height
D_inch = 46 # depth
print("Dimension lengths:", W_inch, H_inch, D_inch)
WHD_inches = [44, 32, 46]
print("Dimension lengths in a list:", WHD_inches)

You may use the `[]` operator to access the individual elements in the list container:

In [ ]:
print("Elements in the list:", WHD_inches[0], WHD_inches[1], WHD_inches[2])

You can add or remove values from a list:

In [ ]:
WHD_inches = [44, 32, 46]
WHD_inches.append(42)
print("42 is append to the end of WHD_inches:", WHD_inches)
WHD_inches.insert(1, 42)
print("42 is inserted in the middle of WHD_inches:", WHD_inches)
del WHD_inches[1]
print("42 is deleted by index:", WHD_inches)
WHD_inches.remove(42)
print("42 is removed from WHD_inches:", WHD_inches)

Python provides many helpers for containers.  You can sort the values:

In [ ]:
print(sorted(WHD_inches))
print(WHD_inches)

Alternate sorting helper can sort the list in place:

In [ ]:
WHD_inches_2 = WHD_inches[:]
WHD_inches_2.sort()
print(WHD_inches_2)

Unlike simple values, adding two lists produces a concatenated list:

In [ ]:
print(W_inch + H_inch + D_inch)
print(WHD_inches + sorted(WHD_inches))

Since adding a value to a list doesn't make sense, Python doesn't allow it:

In [ ]:
W_inch + WHD_inches # This errors out.

## Dictionary

A dictionary is another type of useful container.  It is a key-value pair, and you may construct it using `{}`.

In [ ]:
mapping = {"age": 17, "gender": "unknown"}

print(mapping)

Dictionary is very useful, but this session hasn't gone to the point, so we'll stop here.

# Arrays

Arrays are the best tools to manage homogeneous data.  The [numpy](http://www.numpy.org/) library provides everything we need for arrays in Python.  To create an array, we may use a list as the initial data:

In [ ]:
# Import the numpy library. It's a world-wide convention to alias it to "np".
import numpy as np

# Make a list of integers.
lst = [1, 1, 2, 3, 5]
print('A list:', lst)

# Make an array from the sequence.
array = np.array(lst)
print('An array:', np.array(array))

The `sequence` and the `array` _look differently_.  But their differences are more than skin deep.

In [ ]:
print(type(lst))
print(type(array))
print(array.dtype)

Python knows that the sequence and the array are of different types.  Further, the array has a field `dtype` that indicates what kind of data it holds.

In [ ]:
real_sequence = [1.0, 1.0, 2.0, 3.0, 5.0]
print(real_sequence, type(real_sequence))
real_array = np.array(real_sequence)
print(real_array, type(real_array), real_array.dtype)

A Python list doesn't know the type it contains, but an array does.  This will come in handy for many applications that process large amount of data.

## Creating Arrays

The most straight-forward way to creating arrays is to use the following helpers:

In [ ]:
empty_array = np.empty(10)
print("It will contain garbage, but it doesn't waste time to initialize:", empty_array)

zeroed_array = np.zeros(10)
print("The contents are cleared with zeros:", zeroed_array)

unity_array = np.ones(10)
print("Instead of zeros, fill it with ones:", unity_array)

print("All of their data types are float64 (double-precision floating-point):",
      empty_array.dtype, zeroed_array.dtype, unity_array.dtype)

And more:

In [ ]:
filled_array = np.full(10, 7)
print("Build an array populated with an arbitrary value:", filled_array)

filled_real_array = np.full(10, 7.0)
print("Build an array populated with an arbitrary real value:", filled_real_array)

empty_array = np.empty(10)
empty_array.fill(7)
print("It's the same as creating an empty array and fill the value:", empty_array)

ranged_array = np.arange(10)
print("Build an array with range:", ranged_array)

ranged_real_array = np.arange(10.0)
print("Build with real range:", ranged_real_array)

Some special helpers focus on the content value of the array.  They will save you sometime writing the correct code to determine the boundary values.

In [ ]:
linear_array = np.linspace(0, 2, num=10)
print("Create an equally-spaced array with 10 elements:", linear_array)

## Boolean Arrays

In Numpy, the Boolean arrays are often used to filter wanted or unwanted elements in another array.

In [ ]:
less_than_5 = ranged_array < 5
print("The mask for less than 5:", less_than_5)
print("The values that are less than 5", ranged_array[less_than_5])

all_on_mask = np.ones(10, dtype='bool')
print("All on mask:", all_on_mask)

all_off_mask = np.zeros(10, dtype='bool')
print("All off mask:", all_off_mask)

## Slicing Array

Slicing is a different way to view an array.

In [ ]:
array = np.arange(10)
print("This is the original array:", array)

sub_array = array[:5]
print("This is the sub-array:", sub_array)

sub_array[:] = np.arange(4, -1, -1)
print("The sub-array is changed:", sub_array)

print("And the original array is changed too (!):", array)

In [ ]:
array = np.arange(10.0)
print("Recreate the original array to show how to avoid this:", array)

# Make a copy from the slice.
sub_array = array[:5].copy()
sub_array[:] = np.arange(4, -1, -1)
print("The sub-array is changed, again:", sub_array)
print("But original array remains the same:", array)

## Multi-Dimensional Arrays

So far we only saw one-dimensional arrays, but Numpy wouldn't be so useful if it doesn't do a great job on multi-dimensional arrays.  Multi-dimensional arrays are much more useful than one-dimensional because it's the building-block of matrices and linear algebra.

First let's see how to creating multi-dimensional arrays from one-dimensionals by stacking:

In [ ]:
ranged_array = np.arange(10)
print("A 1-D array:", ranged_array)

hstack_array = np.hstack([ranged_array, ranged_array])
print("Horizontally stacked array:", hstack_array)

vstack_array = np.vstack([ranged_array, ranged_array])
print("Vertically stacked array:", vstack_array)

When the arrays are in multiple dimension, you can specify the axis of operation.  Let's see a simple matrix:

\begin{align*}
A = \left(\begin{array}{ccc}
a_{00} & a_{01} & a_{02} \\
a_{10} & a_{11} & a_{12}
\end{array}\right)
= \left(\begin{array}{ccc}
0 & 1 & 2 \\
3 & 4 & 5
\end{array}\right)
\end{align*}

Since we are talking about Numpy, the indices start from 0 (normal math uses 1-based indexing).  Let's say we sum the elements along the 0th-axis, we'll get:

\begin{align*}
A_{\mathrm{along } 0} = \left(\begin{array}{ccc}
a_{00} + a_{10} & a_{01} + a_{11} & a_{02} + a_{12}
\end{array}\right)
= \left(\begin{array}{ccc}
3 & 5 & 7
\end{array}\right)
\end{align*}

Do it along the 1st-axis:

\begin{align*}
A_{\mathrm{along } 1} = \left(\begin{array}{cc}
a_{00} + a_{01} + a_{02} & a_{10} + a_{11} + a_{12}
\end{array}\right)
= \left(\begin{array}{ccc}
3 & 12
\end{array}\right)
\end{align*}

Now let's see the code:

In [ ]:
reshaped_array = np.arange(6).reshape((2,3))
print("2-D array reshaped from 1-D array:", reshaped_array)

In [ ]:
print("Summation along 0th axis:", reshaped_array.sum(axis=0))

In [ ]:
print("Summation along 1st axis:", reshaped_array.sum(axis=1))

## Project Step 2: Audio Signals

In the first project step, we get the frequency of each note in the C chord.  But before we can play it, the frequencies need to be turned into temporal signals.  To do it, we first create the time marks needed by the signals:

In [ ]:
duration = 5.0 # Play for 5 seconds.

# Create the time marks.
time = np.linspace(0, duration, num=int(sampling_freq*duration))

By using the frequencies and the temporal array, we can create the sinusoidal signal for the 3 notes:

In [ ]:
c4_data = np.sin(2*np.pi * c4_freq * time)
print(len(c4_data), c4_data)

e4_data = np.sin(2*np.pi * e4_freq * time)
print(len(e4_data), e4_data)

g4_data = np.sin(2*np.pi * g4_freq * time)
print(len(g4_data), g4_data)

# Present Data

It is of course incomplete to create data without see them.  Visualization is important for analysis.  In Python we have a powerful tool called [Matplotlib](https://matplotlib.org/) for showing 2D plots.  To use it, import like the following.

In [ ]:
# Import matplotlib interactive API and alias it.
import matplotlib.pyplot as plt

Let's take the signal of middle C and plot it.

In [ ]:
# Plot the first 1000 data points.
plt.plot(time[0:1000], c4_data[0:1000])

Simple, however, primitive.  Like every plot, we should add the axis labels and units.

In [ ]:
plt.plot(time[0:1000], c4_data[0:1000])
plt.xlim(time[0], time[999]) # Leave no blank in the x-axis.
plt.xlabel("time (sec)")
plt.ylabel("normalized amplitude")
plt.title("C4")

We have three notes to show.  But putting them in the same plot doesn't look good.

In [ ]:
plt.plot(time[0:1000], c4_data[0:1000])
plt.plot(time[0:1000], e4_data[0:1000])
plt.plot(time[0:1000], g4_data[0:1000])
plt.xlim(time[0], time[999]) # Leave no blank in the x-axis.
plt.xlabel("time (sec)")
plt.ylabel("normalized amplitude")

A better way is to line them up in three sub plots.

In [ ]:
fig = plt.figure(figsize=(12.0, 12.0)) # Make the plot larger for more details.

axes_c4 = fig.add_subplot(3, 1, 1)
axes_c4.set_xlim(time[0], time[999])
axes_c4.set_ylabel('C4')
axes_c4.tick_params(axis='x', which='both', labelbottom=False) 
axes_c4.plot(time[0:1000], c4_data[0:1000])

axes_e4 = fig.add_subplot(3, 1, 2)
axes_e4.set_xlim(time[0], time[999])
axes_e4.set_ylabel('E4')
axes_e4.tick_params(axis='x', which='both', labelbottom=False) 
axes_e4.plot(time[0:1000], e4_data[0:1000])

axes_g4 = fig.add_subplot(3, 1, 3)
axes_g4.set_xlim(time[0], time[999])
axes_g4.set_xlabel('time (sec)')
axes_g4.set_ylabel('G4')
axes_g4.plot(time[0:1000], g4_data[0:1000])

Now we really see the different frequencies in time domain.  It's basic, but you may find abundance of examples at: https://matplotlib.org/gallery/index.html.

## Project Step 3: Synthesize the Signals

Now add together the 3 signals and scale up the intensity.  They become the chord we want.

In [ ]:
# Make a list of arrays to prepare for stacking.
data_list = [c4_data, e4_data, g4_data]

# Stack the arrays vertically.
data_stacked = np.vstack(data_list)
print(data_stacked.shape)

# Synthesize the signals by adding them together.
data = data_stacked.sum(axis=0)

# Scale up for the right intensity (volume).
data *= 2**13

# Show the synthesized signal.
plt.plot(time[0:1000], data[0:1000])
plt.xlim(time[0], time[999])
plt.xlabel('time (sec)')
plt.ylabel('amplitude')

# Play it.
display(Audio(data.astype('int16'), rate=sampling_freq))

# Preview Next Session: Play a Song

In [ ]:
def play_numbered(numbered_notation, bpm=90, base_octave=4):
    # Middle C is denoted by C4
    note_map = {'0':' ', '1':'c', '2':'d', '3':'e', '4':'f', '5':'g', '6':'a', '7':'b'}
    opt_symbols = (
        # octaves
        '^', 'v',
        # accidentals
        '#', 'b',
        # note length
        '-', '.', '_',
        # misc
        '|', ' ', ','
    )
    beat_sec = 60.0 / bpm
    signals = []
    # init numbered
    default_num = lambda: (base_octave, '', 1.0)
    octave, accidental, nbeat = default_num()
    for it, num in enumerate(numbered_notation):
        # raise/lower octave
        if num == '^':
            octave += 1
        elif num == 'v':
            octave -= 1
        
        # accidentals
        if num in '#b':
            accidental = num
        
        if num in opt_symbols:
            continue
            
        for ahead in numbered_notation[it+1:]:
            if '-' == ahead:
                nbeat += 1
            elif '.' == ahead:
                nbeat += 0.5
            elif '_' == ahead:
                nbeat /= 2
            else:
                break
        
        time = time_data(beat_sec * nbeat, sampling_freq)
        note = note_map[num]
        if note != ' ':
            note = note + accidental + str(octave)
        freq = note_freq(note)
        signal = sine_data(freq, time, damp=beat_sec*0.1)
        signals.append(signal)
        
        # reset to default
        octave, accidental, nbeat = default_num()
        
    data = np.hstack(signals)
    display(Audio(data.astype('int16'), rate=sampling_freq))
    
print("Twinkle Twinkle Little Star:")
play_numbered('1155|665-|4433|221-|5544|332-|5544|332-|1155|665-|4433|221-')

print("Hänschen Klein:")
play_numbered('533-|422-|1234|555-|533-|422-|1355|3--0'
              '2222|234-|3333|345-|533-|422-|1355|1--0')

print("Hänschen Klein at 120 BPM:")
play_numbered('533-|422-|1234|555-|533-|422-|1355|3--0'
              '2222|234-|3333|345-|533-|422-|1355|1--0', bpm=120)

print("Marshmello - TELL ME as 142 BPM:")
play_numbered('#5#1-#5|4-#1_#2_4|#5#1-^#1|^1-#5_#6_4' * 2, bpm=142)